In [1]:
import sys
import os
import natsort
import numpy as np
import pickle

In [2]:
out_dir = "/home/amridul/Documents/forward_project/data/Yumi_Sim"
in_dir = "/home/amridul/Documents/forward_project/data/Yumi_Sim"

In [3]:
subjects = os.listdir(in_dir)
print(subjects)

['S3', 'S1', 'S2']


In [4]:
def dissectKinematicSequence(sub, trial, look_ahead):
    fileNameKin = sub+'_'+trial+'_kinematics.txt'
    fileNameAnnotate = sub+'_'+trial+'color_annot.txt'
    
    print(fileNameKin, fileNameAnnotate)
    
    subDir = os.path.join(in_dir, sub)
    
    surgemeKeys = ['startFrame', 'endFrame', 'label', 'isSuccessful', 'domHand', 'param']
    surgemeInfoDict = {}
    for key in surgemeKeys:
        surgemeInfoDict[key] = []
    
    totalSegments = 0
    with open(os.path.join(subDir, fileNameAnnotate)) as fAnnotate:
        for line in fAnnotate:
            totalSegments += 1
            line = line.replace('\n', '')
            surgemeParts = line.split(' ')
            print(surgemeParts)
            for key, part in zip(surgemeKeys, surgemeParts):
                if key == 'label':
                    surgemeInfoDict[key].append(int(part[1]))
                elif key == 'isSuccessful':
                    surgemeInfoDict[key].append(int(bool(part)))
                else:
                    surgemeInfoDict[key].append(int(part))
                    
            if(surgemeInfoDict['label'][-1] in [1, 6]):
                assert(surgemeInfoDict['param'][-1] != -1)
                assert(surgemeInfoDict['domHand'][-1] != -1)
            elif(surgemeInfoDict['label'][-1] in [2, 3, 7]):
                assert(surgemeInfoDict['param'][-1] == -1)
                assert(surgemeInfoDict['domHand'][-1] != -1)
            elif(surgemeInfoDict['label'][-1] in [4, 5]):
                assert(surgemeInfoDict['param'][-1] == -1)
                assert(surgemeInfoDict['domHand'][-1] == -1)
            else:
                print(surgemeInfoDict['label'][-1])
                assert(0)
                
            if(surgemeInfoDict['label'][-1] == 6):
                surgemeInfoDict['param'][-1] = -1

    X = []
    Y = []
    T_kin = []
    kinLines = open(os.path.join(subDir, fileNameKin)).readlines()
    for frame, line in enumerate (kinLines):
        segment = 0
        while segment < totalSegments:
            if(frame in range(surgemeInfoDict['startFrame'][segment], surgemeInfoDict['endFrame'][segment]+1)):
                break
            segment += 1
        
        parts = line.split(' ')
            
        x = []
        for part in parts[1:]:
            x.append(float(part))
                    
        X.append(x)
        
        if (frame + look_ahead < len(kinLines)): 
            t_line = kinLines[frame+look_ahead] 
        else: 
            t_line = kinLines[-1]

        t_kin = []
        parts = t_line.split(' ')
        for part in parts[12:15]:
            t_kin.append(float(part))
        for part in parts[27:30]:
            t_kin.append(float(part))
            
        T_kin.append(t_kin)
        
        y = []
        if segment < totalSegments:
            for key in surgemeKeys[2:]:
                y.append(surgemeInfoDict[key][segment])
        else:
            y.append(0)

            for key in surgemeKeys[3:]:
                y.append(-1)

        Y.append(y)
    
    return X, Y, T_kin        

In [5]:
lineNum = 0
subjects = ['S1']
for sub in subjects:
    trials = ['T' + str(x) for x in range(1, 12+1)]
    
    for trial in trials:
        X, Y, T_kin = dissectKinematicSequence(sub, trial, look_ahead = 0)
        fileNameX = os.path.join(os.path.join(in_dir, sub), sub+'_'+trial+'_frame_kinematic_data.pkl')
        fileNameY = os.path.join(os.path.join(in_dir, sub), sub+'_'+trial+'_frame_surgeme_data.pkl')
        fileNameT_lin = os.path.join(os.path.join(in_dir, sub), sub+'_'+trial+'_frame_target_data.pkl')        
        
        pickle_handle = open(fileNameX, 'wb')
        print(fileNameX)
        pickle.dump(X, pickle_handle)
        pickle_handle.close()
        
        pickle_handle = open(fileNameY, 'wb')
        print(fileNameY)        
        pickle.dump(Y, pickle_handle)
        pickle_handle.close()
        
        pickle_handle = open(fileNameT_lin, 'wb')
        print(fileNameT_lin)        
        pickle.dump(T_kin, pickle_handle)
        pickle_handle.close()        

S1_T1_kinematics.txt S1_T1color_annot.txt
['63', '353', 'S1', 'True', '0', '1', '']
['353', '373', 'S2', 'True', '0', '-1']
['373', '405', 'S3', 'True', '0', '-1']
['467', '525', 'S1', 'True', '0', '1']
['525', '554', 'S2', 'True', '0', '-1']
['554', '579', 'S3', 'True', '0', '-1']
['579', '653', 'S4', 'True', '-1', '-1']
['653', '739', 'S5', 'True', '-1', '-1']
['739', '795', 'S6', 'False', '1', '5']
['795', '838', 'S1', 'True', '1', '1']
['838', '848', 'S2', 'True', '1', '-1']
['848', '867', 'S3', 'True', '1', '-1']
['867', '962', 'S6', 'True', '1', '5']
['962', '989', 'S7', 'True', '1', '-1']
['989', '1045', 'S1', 'True', '0', '0']
['1045', '1081', 'S2', 'True', '0', '-1']
['1081', '1090', 'S3', 'True', '0', '-1']
['1090', '1131', 'S4', 'True', '-1', '-1']
['1131', '1206', 'S5', 'True', '-1', '-1']
['1206', '1227', 'S6', 'True', '1', '4']
['1227', '1365', 'S7', 'True', '1', '-1']
['1368', '1480', 'S1', 'True', '0', '2']
['1480', '1518', 'S2', 'True', '0', '-1']
['1518', '1528', 'S3'